# TODO: 

- Finish preprocessing data

In [1]:
!pip install tensorflow_hub

     |████████████████████████████████| 92kB 697kB/s eta 0:00:011
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
!pip install tensorflow-datasets

     |████████████████████████████████| 3.1MB 4.7MB/s eta 0:00:01
     |████████████████████████████████| 153kB 48.7MB/s eta 0:00:01
     |████████████████████████████████| 61kB 13.0MB/s eta 0:00:01
     |████████████████████████████████| 829kB 45.9MB/s eta 0:00:01
  Created wheel for dill: filename=dill-0.3.1.1-cp36-none-any.whl size=77454 sha256=7240176233640a6572b4c9e614a4c2a5eed1d72e69fedf5cfb1383421a7766c6
  Stored in directory: /home/ec2-user/.cache/pip/wheels/59/b1/91/f02e76c732915c4015ab4010f3015469866c1eb9b14058d8e7
  Created wheel for future: filename=future-0.18.2-cp36-none-any.whl size=491095 sha256=63b1a8c1d1d14185459f125849755be685cfe7b8c4b03b1fe2f8b58aa80700fd
  Stored in directory: /home/ec2-user/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
  Created wheel for promise: filename=promise-2.3-cp36-none-any.whl size=20685 sha256=be2be8db0d6e578ad9cd01d59e25a4c1e91c39f789377da15cbc50ae88602b1f
  Stored in directory: /home/ec2-user/.cache/pip/

In [3]:
!pip install bert-tensorflow

     |████████████████████████████████| 71kB 4.2MB/s eta 0:00:011
You should consider upgrading via the 'pip install --upgrade pip' command.


In [1]:
import tensorflow as tf
#tf.enable_eager_execution()
#tf.executing_eagerly()
import tensorflow.keras as keras
import tensorflow_datasets as tfds
import tensorflow_hub as hub


In [2]:
datasets, info = tfds.load("imdb_reviews", as_supervised=True, with_info=True)

In [3]:
train_size = info.splits['train'].num_examples

In [4]:
info

tfds.core.DatasetInfo(
    name='imdb_reviews',
    version=1.0.0,
    description='Large Movie Review Dataset.
This is a dataset for binary sentiment classification containing substantially more data than previous benchmark datasets. We provide a set of 25,000 highly polar movie reviews for training, and 25,000 for testing. There is additional unlabeled data for use as well.',
    homepage='http://ai.stanford.edu/~amaas/data/sentiment/',
    features=FeaturesDict({
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
        'text': Text(shape=(), dtype=tf.string),
    }),
    total_num_examples=100000,
    splits={
        'test': 25000,
        'train': 25000,
        'unsupervised': 50000,
    },
    supervised_keys=('text', 'label'),
    citation="""@InProceedings{maas-EtAl:2011:ACL-HLT2011,
      author    = {Maas, Andrew L.  and  Daly, Raymond E.  and  Pham, Peter T.  and  Huang, Dan  and  Ng, Andrew Y.  and  Potts, Christopher},
      title     = {Learning Word

# Preporcessing

In [5]:
from bert import tokenization
import bert

# This is a path to an uncased (all lowercase) version of BERT
BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"

def create_tokenizer_from_hub_module():
    """Get the vocab file and casing info from the Hub module."""
    with tf.Graph().as_default():
        bert_module = hub.Module(BERT_MODEL_HUB)
        tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
        with tf.Session() as sess:
            vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                            tokenization_info["do_lower_case"]])
      
    return bert.tokenization.FullTokenizer(
      vocab_file=vocab_file, do_lower_case=do_lower_case)

tokenizer = create_tokenizer_from_hub_module()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [9]:
tokenizer.tokenize("This here's an example of using the BERT tokenizer")

['this',
 'here',
 "'",
 's',
 'an',
 'example',
 'of',
 'using',
 'the',
 'bert',
 'token',
 '##izer']

In [6]:
def convert_sentence_to_features(sentence, tokenizer, max_seq_len):
    tokens = ['[CLS]']
    tokens.extend(tokenizer.tokenize(sentence))
    if len(tokens) > max_seq_len-1:
        tokens = tokens[:max_seq_len-1]
    tokens.append('[SEP]')
    
    segment_ids = [0] * len(tokens)
    input_ids = tokenizer.convert_tokens_to_ids(tokens)
    input_mask = [1] * len(input_ids)

    #Zero Mask till seq_length
    zero_mask = [0] * (max_seq_len-len(tokens))
    input_ids.extend(zero_mask)
    input_mask.extend(zero_mask)
    segment_ids.extend(zero_mask)
    
    return input_ids, input_mask, segment_ids

def convert_sentences_to_features(sentences, tokenizer, max_seq_len=20):
    all_input_ids = []
    all_input_mask = []
    all_segment_ids = []
    
    for sentence in sentences:
        input_ids, input_mask, segment_ids = convert_sentence_to_features(sentence, tokenizer, max_seq_len)
        all_input_ids.append(input_ids)
        all_input_mask.append(input_mask)
        all_segment_ids.append(segment_ids)
    
    return all_input_ids, all_input_mask, all_segment_ids

In [62]:
sentence = """This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.">, <tf.Tensor: id=20482, shape=(), dtype=int64, numpy=0>)
(<tf.Tensor: id=20483, shape=(), dtype=string, numpy=b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the film was rubbish. The plot development was constant. Constantly slow and boring. Things seemed to happen, but with no explanation of what was causing them or why. I admit, I may have missed part of the film, but i watched the majority of it and everything just seemed to happen of its own accord without any real concern for anything else. I cant recommend this film at all."""

In [71]:
convert_sentence_to_features(sentence, tokenizer, 20)

([101,
  2023,
  2001,
  2019,
  7078,
  6659,
  3185,
  1012,
  2123,
  1005,
  1056,
  2022,
  26673,
  1999,
  2011,
  5696,
  3328,
  2368,
  2030,
  102],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [73]:
convert_sentences_to_features([sentence], tokenizer)

([[101,
   2023,
   2001,
   2019,
   7078,
   6659,
   3185,
   1012,
   2123,
   1005,
   1056,
   2022,
   26673,
   1999,
   2011,
   5696,
   3328,
   2368,
   2030,
   102]],
 [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
 [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [60]:
for d in datasets['train'].take(4):
    print(d)

(<tf.Tensor: id=20481, shape=(), dtype=string, numpy=b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.">, <tf.Tensor: id=20482, shape=(), dtype=int64, numpy=0>)
(<tf.Tensor: id=20483, shape=(), dtype=string, numpy=b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired,

In [7]:
def encoding(x):
    return convert_sentences_to_features([x.numpy().decode('utf-8')], tokenizer)

dataset_encoded = datasets['train'].map(lambda X, y: (tf.py_function(func=encoding, inp=[X], Tout=(tf.int16, tf.int16, tf.int16)), y))

In [79]:
for X_batch in dataset_encoded.take(4):
    print(X_batch)

(<tf.Tensor: id=20613, shape=(3, 1, 20), dtype=int16, numpy=
array([[[  101,  2023,  2001,  2019,  7078,  6659,  3185,  1012,  2123,
          1005,  1056,  2022, 26673,  1999,  2011,  5696,  3328,  2368,
          2030,   102]],

       [[    1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0]]], dtype=int16)>, <tf.Tensor: id=20614, shape=(), dtype=int64, numpy=0>)
(<tf.Tensor: id=20619, shape=(3, 1, 20), dtype=int16, numpy=
array([[[ 101, 1045, 2031, 2042, 2124, 2000, 2991, 6680, 2076, 3152,
         1010, 2021, 2023, 2003, 2788, 2349, 2000, 1037, 5257,  102]],

       [[   1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1]],

       [[   0,  

In [8]:
train_set = dataset_encoded.batch(32).prefetch(1)

In [81]:
for X_batch, y in train_set.take(1):
    print(X_batch)

tf.Tensor(
[[[[  101  2023  2001 ...  2368  2030   102]]

  [[    1     1     1 ...     1     1     1]]

  [[    0     0     0 ...     0     0     0]]]


 [[[  101  1045  2031 ...  1037  5257   102]]

  [[    1     1     1 ...     1     1     1]]

  [[    0     0     0 ...     0     0     0]]]


 [[[  101 10856  7008 ... 13962  2507   102]]

  [[    1     1     1 ...     1     1     1]]

  [[    0     0     0 ...     0     0     0]]]


 ...


 [[[  101  2023  3185 ...  2006  4966   102]]

  [[    1     1     1 ...     1     1     1]]

  [[    0     0     0 ...     0     0     0]]]


 [[[  101  3505 10184 ...  6424  1000   102]]

  [[    1     1     1 ...     1     1     1]]

  [[    0     0     0 ...     0     0     0]]]


 [[[  101  1006  9826 ... 25230  2075   102]]

  [[    1     1     1 ...     1     1     1]]

  [[    0     0     0 ...     0     0     0]]]], shape=(32, 3, 1, 20), dtype=int16)


# Model

In [9]:
bert = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"

bert_module = hub.Module(bert, trainable=False, 
   name='bexrt')


In [10]:


# Build model
max_seq_length = 20
in_id = tf.keras.layers.Input(shape=(max_seq_length,), name="input_ids")
in_mask = tf.keras.layers.Input(shape=(max_seq_length,), name="input_masks")
in_segment = tf.keras.layers.Input(shape=(max_seq_length,), name="segment_ids")



bert_inputs = dict(
      input_ids=in_id,
      input_mask=in_mask,
      segment_ids=in_segment)

bert_outputs = bert_module(
      inputs=bert_inputs,
      signature="tokens",
      as_dict=True)

# Use "pooled_output" for classification tasks on an entire sentence.
# Use "sequence_outputs" for token-level output.
output_layer = bert_outputs["pooled_output"]

hidden_size = output_layer.shape[-1].value

hidden = keras.layers.Dense(128, activation="relu")(output_layer)
output = keras.layers.Dense(1, activation="sigmoid")(hidden)


model = keras.Model(inputs=[in_id, in_mask, in_segment], outputs=[output])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(
train_set,
    epochs=1,
    batch_size=32
)

model = tf.keras.models.Model(inputs=bert_inputs, outputs=pred)


ValueError: Tensor conversion requested dtype int32 for Tensor with dtype float32: 'Tensor("input_ids:0", shape=(?, 20), dtype=float32)'

In [ ]:
# Build model
max_seq_length = 20
in_id = tf.keras.layers.Input(shape=(max_seq_length,), name="input_ids")
in_mask = tf.keras.layers.Input(shape=(max_seq_length,), name="input_masks")
in_segment = tf.keras.layers.Input(shape=(max_seq_length,), name="segment_ids")

bert_inputs = [in_id, in_mask, in_segment]

# Instantiate the custom Bert Layer defined above
bert_output = BertLayer(n_fine_tune_layers=10)(bert_inputs)

# Build the rest of the classifier 
dense = tf.keras.layers.Dense(256, activation='relu')(bert_output)
pred = tf.keras.layers.Dense(1, activation='sigmoid')(dense)

model = tf.keras.models.Model(inputs=bert_inputs, outputs=pred)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(
train_set,
    validation_data=([test_input_ids, test_input_masks, test_segment_ids], test_labels),
    epochs=1,
    batch_size=32
)

In [82]:
bert_path = BERT_MODEL_HUB
from keras import backend as K

class BertLayer(tf.layers.Layer):
    def __init__(self, n_fine_tune_layers=10, **kwargs):
        self.n_fine_tune_layers = n_fine_tune_layers
        self.trainable = True
        self.output_size = 768
        super(BertLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.bert = hub.Module(
            bert_path,
            trainable=self.trainable,
            name="{}_module".format(self.name)
        )
        trainable_vars = self.bert.variables
        
        # Remove unused layers
        trainable_vars = [var for var in trainable_vars if not "/cls/" in var.name]
        
        # Select how many layers to fine tune
        trainable_vars = trainable_vars[-self.n_fine_tune_layers :]
        
        # Add to trainable weights
        for var in trainable_vars:
            self._trainable_weights.append(var)
        
        # Add non-trainable weights
        for var in self.bert.variables:
            if var not in self._trainable_weights:
                self._non_trainable_weights.append(var)
        
        super(BertLayer, self).build(input_shape)

    def call(self, inputs):
        inputs = [K.cast(x, dtype="int32") for x in inputs]
        input_ids, input_mask, segment_ids = inputs
        bert_inputs = dict(
            input_ids=input_ids, input_mask=input_mask, segment_ids=segment_ids
        )
        result = self.bert(inputs=bert_inputs, signature="tokens", as_dict=True)[
            "pooled_output"
        ]
        return result

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_size)


In [83]:
# Build model
max_seq_length = 20
in_id = tf.keras.layers.Input(shape=(max_seq_length,), name="input_ids")
in_mask = tf.keras.layers.Input(shape=(max_seq_length,), name="input_masks")
in_segment = tf.keras.layers.Input(shape=(max_seq_length,), name="segment_ids")

bert_inputs = [in_id, in_mask, in_segment]

# Instantiate the custom Bert Layer defined above
bert_output = BertLayer(n_fine_tune_layers=10)(bert_inputs)

# Build the rest of the classifier 
dense = tf.keras.layers.Dense(256, activation='relu')(bert_output)
pred = tf.keras.layers.Dense(1, activation='sigmoid')(dense)

model = tf.keras.models.Model(inputs=bert_inputs, outputs=pred)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(
train_set,
    validation_data=([test_input_ids, test_input_masks, test_segment_ids], test_labels),
    epochs=1,
    batch_size=32
)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


TypeError: The following are legacy tf.layers.Layers:
  <__main__.BertLayer object at 0x7f2aabc1ad68>
To use keras as a framework (for instance using the Network, Model, or Sequential classes), please use the tf.keras.layers implementation instead. (Or, if writing custom layers, subclass from tf.keras.layers rather than tf.layers)

# Benchmark
http://nlpprogress.com/english/sentiment_analysis.html

In [ ]:
embed_size = 128

model = keras.models.Sequential([
    keras.layers.Embedding(vocab_size + num_oov_buckets, embed_size, input_shape=[None], mask_zero=True),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.GRU(128,),
    keras.layers.Dense(1, activation="sigmoid")
])
batch_size=32
num_epochs=15
steps_per_epoch_train = train_size // num_epochs // batch_size
steps_per_epoch_test = train_size // num_epochs // batch_size

model.compile(loss='binary_crossentropy', optimizer="adam", metrics=["accuracy"])
history = model.fit(train_set, epochs=num_epochs, steps_per_epoch=steps_per_epoch)

# TensorBoard?

# Pretrained embedings

In [3]:
batch_size=32
num_epochs=15

datasets, info = tfds.load("imdb_reviews", as_supervised=True, with_info=True)
                           
train_size = info.splits["train"].num_examples
train_set = datasets["train"].repeat().batch(batch_size).prefetch(1)

test_size = info.splits["test"].num_examples
test_set = datasets["test"].repeat().batch(batch_size).prefetch(1)


                           
steps_per_epoch_train = train_size // num_epochs // batch_size
steps_per_epoch_test = train_size // num_epochs // batch_size

In [4]:
for element in train_set.take(1):
    print(element)

(<tf.Tensor: id=212, shape=(32,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasio

In [5]:
import tensorflow_hub as hub

## nnlm-en-dim50

In [6]:
embed = hub.load("https://tfhub.dev/google/tf2-preview/nnlm-en-dim50/1")
embeddings = embed(["cat is on the mat", "dog is in the fog"])

In [7]:
embeddings

<tf.Tensor: id=396, shape=(2, 50), dtype=float32, numpy=
array([[ 0.16589954,  0.0254965 ,  0.1574857 ,  0.17688066,  0.02911299,
        -0.03092718,  0.19445257, -0.05709129, -0.08631689, -0.04391516,
         0.13032274,  0.10905275, -0.08515751,  0.01056632, -0.17220995,
        -0.17925954,  0.19556305,  0.0802278 , -0.03247919, -0.49176937,
        -0.07767699, -0.03160921, -0.13952136,  0.05959712,  0.06858718,
         0.22386682, -0.16653948,  0.19412343, -0.05491862,  0.10997339,
        -0.15811177, -0.02576607, -0.07910853, -0.258499  , -0.04206644,
        -0.20052543,  0.1705603 , -0.15314153,  0.0039225 , -0.28694248,
         0.02468278,  0.11069503,  0.03733957,  0.01433943, -0.11048374,
         0.11931834, -0.11552787, -0.11110869,  0.02384969, -0.07074881],
       [ 0.1437864 ,  0.08291595,  0.10897306,  0.04464385, -0.03630389,
        -0.12605834,  0.20263346,  0.12862863, -0.07873426, -0.01195358,
         0.0020956 , -0.03080653, -0.08019945, -0.18797135, -0.119

# Tensorboard

In [13]:
import datetime

In [14]:
log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [10]:
import tensorflow_hub as hub

model = keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/tf2-preview/nnlm-en-dim50/1",
                   dtype=tf.string, input_shape=[], output_shape=[50]),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dense(1, activation="sigmoid")
])
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 50)                48190600  
_________________________________________________________________
dense (Dense)                (None, 128)               6528      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 48,197,257
Trainable params: 6,657
Non-trainable params: 48,190,600
_________________________________________________________________


In [29]:
steps_per_epoch_train

52

In [15]:
history = model.fit(train_set, steps_per_epoch=steps_per_epoch_train, epochs=num_epochs, callbacks=[tensorboard_callback])

Train for 52 steps
Epoch 1/15


 1/52 [..............................] - ETA: 30s - loss: 0.6005 - acc: 0.6875WARNING:tensorflow:Method (on_train_batch_end) is slow compared to the batch update (0.277269). Check your callbacks.


52/52 [==============================] - 1s 24ms/step - loss: 0.5317 - acc: 0.7308
Epoch 2/15
52/52 [==============================] - 1s 12ms/step - loss: 0.5084 - acc: 0.7578
Epoch 3/15
52/52 [==============================] - 1s 12ms/step - loss: 0.5270 - acc: 0.7362
Epoch 4/15
52/52 [==============================] - 1s 12ms/step - loss: 0.5145 - acc: 0.7410
Epoch 5/15
52/52 [==============================] - 1s 12ms/step - loss: 0.5191 - acc: 0.7398
Epoch 6/15
52/52 [==============================] - 1s 12ms/step - loss: 0.4961 - acc: 0.7620
Epoch 7/15
52/52 [==============================] - 1s 12ms/step - loss: 0.5161 - acc: 0.7446
Epoch 8/15
52/52 [==============================] - 1s 12ms/step - loss: 0.5093 - acc: 0.7596
Epoch 9/15
52/52 [==============================] - 1s 12ms/step - loss: 0.5182 - acc: 0.7440
Epoch 10/15
52/52 [==============================] - 1s 12ms/step - loss: 0.4950 - acc: 0.7698
Epoch 11/15
52/52 [==============================] - 1s 12ms/step - lo

In [16]:
model.evaluate(test_set, steps=steps_per_epoch_test)

52/52 [==============================] - 1s 12ms/step - loss: 0.5405 - acc: 0.7302


[0.5405245767189906, 0.7301683]

In [24]:
!kill 13518

In [27]:
%reload_ext tensorboard

In [29]:
%load_ext tensorboard
%tensorboard --logdir logs/fit  --host localhost

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
